In [4]:
from bs4 import BeautifulSoup
import requests 
import re
from time import sleep
import pandas as pd
import time
from tqdm import tqdm
import pickle
import itertools
import random
with open('csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
#リストを一緒くたに一つに
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))




#flattened_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
with open('csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)
flatclosing_urls = list(itertools.chain(ex_data.values()))

#-------------------------------------------------------------------------一先ず終了2023/01/05----------------------------------------------------------------------#
#上記を満たした企業コードの決算ページxxxxxx/resultsにアクセスし企業名と条件の財務の値を取得
focus_url=[]
text1=[]
for flatclosing_url in flatclosing_urls[:2]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    decopo=soup.find_all('a')
    for a in decopo:
        a.decompose()
    tagget= soup.find_all('h2')
    text1.append(tagget)
text1_stringso = [[h2 for h2 in text] for text in text1]
        
target_h2_stringso = [[t for t in ext if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or
                      "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t] for ext in text1_stringso ]
test_lists = [[i.find_next_sibling().find_all('span', class_='text') for i in uu]for uu in target_h2_stringso   if len(uu)==8]

#-------------------------------------------------------------------------ひとまず終了き------------------------------------------------------------------------#
d_list=[]
for test_list in test_lists:
    if len(test_list)==8:
            sales_list=[[sale.text for sale in test_list[0]]for test_list in test_lists]


            #zfill完了
            eval_sale_list = []
            for sale in sales_list:
                eval_sale_list.append(convert_sales_figure(sale))
            #採点完了
            uriagedaka_stock = check_uriagedaka(eval_sale_list)


#-------------------------------------------------------------------------2:ここからEPS2023/01/05--------------------------------------------------------------------------#
    eps_list=[ep.text for ep in test_lists[1]]
    #zfill完了
    zeps_list = []
    for ep in eps_list:
        zeps_list.append(convert_eps(ep))
    eps_stock = check_eps(zeps_list)

#-------------------------------------------------------------------------3:ここから営業利益率2023/01/05- -------------------------------------------------------------------------#     

    op_list=[op.text for op in test_lists[2]]
    zop_list=[]
    for s in op_list:
        hennkan = s.split('%')[0]  # '15.65' のような文字列を取得
        zop_list.append(eval(hennkan))

    eiri_stock = check_eiri(zop_list)


#-------------------------------------------------------------------------4:ここから自己資本利益率2023/01/05-------------------------------------------------------------------------#

    operation_list=[operation.text for operation in test_lists[3]]
    zoption_list=[]
    for s in operation_list:
        result = floatparcent_hennkan(s)  # hennkan 関数を呼び出す
        zoption_list.append(result)

        zikosihon_stock = check_zikosihon(zoption_list)


               #zoption_list 変数に格納された要素を、個別に確認する
    #     for i, value in enumerate(zoption_list):
    #         # value 変数が 40.00 以上であるかどうかを判定する
    #         if float(40.00) <= float(value):
    #             print(f'index {i}: OK')
    #         else:
    #             print(f'index {i}: NOT')
#-------------------------------------------------------------------------5:ここから現金2023/01/05-------------------------------------------------------------------------#
    cash_list=[cash.text for cash in test_lists[5]]
      #zfill完了
    eval_cash_list = []
    for cash in cash_list:
        eval_cash_list.append(convert_sales_figure(cash))
        genkin_stock = check_genkin(eval_cash_list)
       #-------------------------------------------------------------------------6:ここからCF -------------------------------------------------------------------------#

    cf_flow_list=[cfflow.text for cfflow in test_lists[4]]
    #zfill完了
    eval_cf_flow_list = []
    for flow in cf_flow_list:
        eval_cf_flow_list.append(convert_flow_figure(flow))
    # モジュール内の関数を呼び出す
        cflow_stock = check_cflow_order(eval_cf_flow_list)

    #-------------------------------------------------------------------------7:ここから一株当たり-------------------------------------------------------------------------#
        #zfill完了

    stock_list=[stock.text for stock in test_lists[6]]

    zstock_list=[]
    for s in stock_list:
        result = convert_stock_figure(s)  # hennkan 関数を呼び出す
        zstock_list.append(result)   
        
    hitokabu_stock=check_zstock_order(zstock_list)

        #-------------------------------------------------------------------------8:配当性向-------------------------------------------------------------------------#
    return_list=[return_money.text for return_money in test_lists[7]]
    #zfill完了
    zreturn_list = []
    for return_money in return_list:
        zreturn_list.append(convert_return(return_money))

    seikou_stock=check_zreturn_range(zreturn_list)
        #-------------------------------------------------------------------------ひとまず終了------------------------------------------------------------------------#
    analyze=[]
    analyze.append(title)
    analyze.append(uriagedaka_stock)
    analyze.append(eps_stock)
    analyze.append(eiri_stock)
    analyze.append(zikosihon_stock)
    analyze.append(genkin_stock)
    analyze.append(cflow_stock)
    analyze.append(hitokabu_stock)
    analyze.append(seikou_stock)
    
    assess=[]
    d_list_dic={
        '会社名':title,
        '売上高':uriagedaka_stock,
        'EPS':eps_stock,
        '営業利益':eiri_stock,
        '自己資本比率':zikosihon_stock,
        '現金':genkin_stock,
        '営業キャッシュフロー':cflow_stock,
        '一株配当':hitokabu_stock,
        '配当性向':seikou_stock,
    }
    assess.append(d_list_dic)
    dff=pd.DataFrame(assess)
    dff.to_csv('irbank/csv/stock_assess.csv',index=True,encoding='utf-8-sig',mode='a')
    


    fine=[]
    for analy in analyze:
        if 'OK'in analy or len(analy)>=0:#analyがOKかNOTが２以下なら◯
            fine.append('○')
    #     if len(analy)==1:
    #         fine.append('○')
        else:
            fine.append('X')#NOT２以上なら×
    if fine.count('○') >= 0:#もし◯が１より上ならd＿list
        d_list.append(title)
        d_list.append(fine)
        print(f"{len(d_list)}個入りました")
        df=pd.DataFrame(d_list)
        #to_csv()を使って、データフレームをCSV出力する
        df.to_csv('irbank/csv/stock_target.csv',index=True,encoding='utf-8-sig',mode='a')
else:
    pass


733


TypeError: expected string or bytes-like object

In [7]:
focus_url

['https://irbank.net/E04236/results', 'https://irbank.net/E04235/results']

In [1162]:
focus_url=[]
text=[]
for flatclosing_url in flatclosing_urls[0:3]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
    text.append(tagget)


KeyboardInterrupt: 

In [9]:
with open('csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
#リストを一緒くたに一つに
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))
#flattened_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
with open('csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)
flatclosing_urls = list(itertools.chain(ex_data.values()))
focus_url=[]
text1=[]
for flatclosing_url in flatclosing_urls[:2]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    decopo=soup.find_all('a')
    for a in decopo:
        a.decompose()
    tagget= soup.find_all('h2')
    text1.append(tagget)
text1_stringso = [[h2 for h2 in text] for text in text1]
        
target_h2_stringso = [[t for t in ext if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or
                      "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t] for ext in text1_stringso ]
test_lists = [[i.find_next_sibling().find_all('span', class_='text') for i in uu]for uu in target_h2_stringso   if len(uu)==8]



# test_lists=[]
# for uu in target_h2_stringso:
#     if len(uu)==8:
#         for i in uu:
#             bro=i.find_next_sibling()
#             ss = bro.find_all('span', class_='text')
#             test_lists.append(ss)
        
        
        
#     for tag in soup.find_all('h2'):
#         text1.append(tag.text)
# ii = []
# for i in range(len(text1)):
#     ii.append([html.fromstring(t) for t in text1[i]])
# ii=[]
# for tem in text1:
#     for kkc in tem:
#         ii.append(kkc)



733


In [11]:
test_lists[1]

[[<span class="text">2兆5846億</span>,
  <span class="text">2兆4299億</span>,
  <span class="text">1兆6973億</span>,
  <span class="text">1兆9291億</span>,
  <span class="text">1兆8078億</span>,
  <span class="text">1兆8971億</span>,
  <span class="text">2兆2372億</span>,
  <span class="text">2兆4018億</span>,
  <span class="text">2兆2723億</span>,
  <span class="text">1兆9238億</span>,
  <span class="text">2兆1832億</span>,
  <span class="text">1兆8293億</span>,
  <span class="text">1兆6683億</span>,
  <span class="text">1兆6084億</span>,
  <span class="text">2兆2807億</span>,
  <span class="text">2兆7000億</span>],
 [<span class="text">-34.2円</span>,
  <span class="text">153.94円</span>,
  <span class="text">-142.74円</span>,
  <span class="text">37.04円</span>,
  <span class="text">64.95円</span>,
  <span class="text">93.53円</span>,
  <span class="text">35.85円</span>,
  <span class="text">-524.11円</span>,
  <span class="text">39.86円</span>,
  <span class="text">-87.93円</span>,
  <span class="text">61.46円</span>,
  <sp

In [25]:
d_list=[]
for test_list in test_lists:
    if len(test_list)==8:
            sales_list=[[sale.text for sale in test_list[0]]for test_list in test_lists]



In [1272]:
text1_stringso = [[h2.text for h2 in text] for text in text1]


In [26]:
sales_list

[['1兆9456億',
  '1兆8658億',
  '1兆3479億',
  '1兆5436億',
  '1兆4352億',
  '1兆5091億',
  '1兆7294億',
  '1兆8170億',
  '1兆7122億',
  '1兆5043億',
  '1兆6523億',
  '1兆2340億',
  '1兆1554億',
  '9914億2600万',
  '1兆2693億',
  '1兆6000億'],
 ['2兆5846億',
  '2兆4299億',
  '1兆6973億',
  '1兆9291億',
  '1兆8078億',
  '1兆8971億',
  '2兆2372億',
  '2兆4018億',
  '2兆2723億',
  '1兆9238億',
  '2兆1832億',
  '1兆8293億',
  '1兆6683億',
  '1兆6084億',
  '2兆2807億',
  '2兆7000億']]

In [1205]:
focus_url=[]
text1=[]
for flatclosing_url in flatclosing_urls[:2]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    decopo=soup.find_all('a')
    for a in decopo:
        a.decompose()
    for tag in soup.find_all('h2'):
        text1.append(tag.text).    flatclosing_urlsの１と2のそれぞれをtext1[0],text1[1]に入れたtext1にしたいのですが
​
for tem in ii:
    print(type(tem))
# target_h2 = []
# for t in ii:
#     if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t:
#         target_h2.append(t)

In [1206]:
print(type(tem))

<class 'bs4.element.ResultSet'>


In [1207]:
ii[0]

IndexError: list index out of range

In [1137]:

for tem in tagget[2]:
    print(tem)
    print(type(tem))

経常利益
<class 'bs4.element.NavigableString'>


In [1106]:
text = []#正規は８項目  
for element in list_tagget:
    for i in range(len(element)):
        ele = element[i]
        for a in ele.find_all('a'):
            a.decompose()
            text.append(element)

        


In [1111]:
focus_url=[]
list_tagget=[]
for flatclosing_url in flatclosing_urls[0:3]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
    list_tagget.append(tagget)

In [1139]:
tagget[0]

<h2>売上高</h2>

In [6]:
flatclosing_urls

['/E04236/results',
 '/E04235/results',
 '/E04239/results',
 '/E00037/results',
 '/E04288/results',
 '/E00041/results',
 '/E00060/results',
 '/E04237/results',
 '/E00492/results',
 '/E01098/results',
 '/E01152/results',
 '/E01136/results',
 '/E01076/results',
 '/E04806/results',
 '/E01259/results',
 '/E03988/results',
 '/E02210/results',
 '/E31946/results',
 '/E02322/results',
 '/E27281/results',
 '/E31054/results',
 '/E01738/results',
 '/E00120/results',
 '/E00708/results',
 '/E00083/results',
 '/E04072/results',
 '/E03333/results',
 '/E35631/results',
 '/E00442/results',
 '/E01759/results',
 '/E02561/results',
 '/E02155/results',
 '/E04243/results',
 '/E03733/results',
 '/E02464/results',
 '/E04048/results',
 '/E01402/results',
 '/E02802/results',
 '/E02343/results',
 '/E33616/results',
 '/E04069/results',
 '/E03259/results',
 '/E35487/results',
 '/E31979/results',
 '/E02281/results',
 '/E04426/results',
 '/E02838/results',
 '/E33603/results',
 '/E02503/results',
 '/E02671/results',


In [660]:
with open('irbank/csv/closing_dict.pickle', 'wb') as f:
            pickle.dump(ex_data, f)
flatclosing_urls = list(itertools.chain(ex_data.values()))

In [335]:
with open('irbank/csv/securities_codes.pickle','rb') as f:
    securities_codes=pickle.load(f)

In [78]:
def convert_eps(eps_figure: str) -> int:
    number_str = eps_figure.split('円')[0]
    hennkan = number_str + '*100'
    return eval(hennkan)

In [77]:
def convert_sales(*args):
    eval_list = []
    for arg in args:
        eval_arg = []
        for value in arg:
            # 兆しかない時
            if '兆' in value and '億' not in value and '万' not in value:
                hennkan = value.replace('兆', '*10**12')
                eval_arg.append(eval(hennkan))
        eval_list.append(eval_arg)
    return eval_list


In [3]:


def get_securities_codes(start_page, end_page):
    url='https://minkabu.jp/financial_item_ranking/dividend_yield?page={}'
    dd_list=[]
    for i in range(start_page, end_page+1):
        target_url=url.format(i)
        r=requests.get(target_url)
        sleep(random.uniform(3, 8)) 
        soup=BeautifulSoup(r.text,'html.parser')
        tab=soup.find('table',class_='md_table rnk_li')
        conts=tab.find_all('tr')
        del conts[0]

        for cont in conts:
            rank=cont.find('span',class_='ranking_no fsize_l').text
            code=cont.find('div',class_='md_sub').text
            name=cont.find('div',class_='fwb w90p').text
            y=cont.find('td',class_='tar cur vamd').text
            yeild= y.strip().replace('%', '')
            dic={
                'ランキング':rank,
                '証券コード':code,
                '企業名':name,
                '配当利回り':yeild,
            }
            if float(yeild) >= 3.75:
                dd_list.append(dic)
    securities_codes = [d['証券コード'] for d in dd_list]
    return securities_codes





def float_hennkan(s):
    # *だけの時
    if '*' in s:
        s = s.split('円')[0]  # '15.65' のような文字列を取得
        s = s.replace('*', '')  # * を削除
        if '.'in s:
            integer_part, decimal_part =s.split('.')  # '15' と '65' のような文字列を取得
            integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
            decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
            hennkan_result= f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
    #小数点の時
    elif '.' in s :
        number_str = s.split('円')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
      

    else:
        # 整数部と小数部が分離されていない場合
        number_str = s.split('円')[0]  # '86' のような文字列を取得
        integer_part = number_str.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result
    

#     if '.' in s:
#         number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
#         integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
#         hennkan_result = f"{integer_part1}.00"
#         return hennkan_result
#     *だけの時
#     elif '*' in s:
#         kn = s.split('*')[1]
#         number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
#         integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
#         hennkan_result = f"{integer_part1}.00"
#         return hennkan_result



def convert_sales_figure(sales_figure: str) -> int:
    #兆しかない時
    if '兆' in sales_figure and '億' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12')
        return eval(hennkan)
    #億しかない時
    elif '億' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8')
        return eval(hennkan)
    #万しかない時
    elif '万' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('万', '*10**4')
        return eval(hennkan)
    #兆と億の時
    elif'兆'in sales_figure and '億' in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12+').replace('億','*10**8')
        return eval(hennkan)
    #億と万の時
    elif'億'in sales_figure and '万' in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8+').replace('万','*10**4')
        return eval(hennkan)
    else:
        hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sales_figure)
        return eval(hennkan)



def convert_eps(eps_figure: str) -> int:
    number_str = eps_figure.split('円')[0]
    hennkan = number_str + '*100'
    return eval(hennkan)


def convert_stock_figure(stock_figure: str) -> int:
    if '.' in stock_figure and '*' not in stock_figure :
        number_str = stock_figure.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    elif '*' in stock_figure and '.' not in stock_figure :
        kn = stock_figure.split('*')[1]
        number_str1 = kn .split('円')[0]
        integer_part1 = number_str1.zfill(2)
        return f"{integer_part1}.00"
    
    elif'*'in stock_figure and '.' in stock_figure:
        kn = stock_figure.split('*')[1]
        number_str = kn.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    else:
        number_str = stock_figure.split('円')[0]
        integer_part = number_str.zfill(2)
        return f"{integer_part}.00"
    

def convert_stock(stock_price: str) -> str:
    if '.' in stock_price:
        number_str = stock_price.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    elif '*' in stock_price:
        kn = stock_price.split('*')[1]
        number_str1 = kn .split('円')[0]
        integer_part1 = number_str1.zfill(2)
        return f"{integer_part1}.00"
    else:
        number_str = stock_price.split('円')[0]
        integer_part = number_str.zfill(2)
        return f"{integer_part}.00"



def convert_return(return_figure: str) -> str:
    if '.' in return_figure:
        number_str = return_figure.split('%')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"

    else:
        integer_part = return_figure.split('%')[0]
        integer_part = integer_part.zfill(2)
        return f"{integer_part}.00"

def process_cashes(cashes):
    cash_list = [cash.text for cash in cashes]
    eval_cash_list = []
    for cash in cash_list:
        # 兆しかない時
        if '兆' in cash and '億' not in cash and '万' not in cash:
            hennkan = cash.replace('兆', '*10**12')
            eval_cash_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('億', '*10**8')
            eval_cash_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('万', '*10**4')
            eval_cash_list.append(eval(hennkan))
        # 兆と億の時
        elif '兆' in cash and '億' in cash:
            hennkan = cash.replace('兆', '*10**12+').replace('億','*10**8')
            eval_cash_list.append(eval(hennkan))
        # 億と万の時
        elif '億' in cash and '万' in cash:
            hennkan = cash.replace('億', '*10**8+').replace('万','*10**4')
            eval_cash_list.append(eval(hennkan))
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_cash_list.append(eval(hennkan))
    return eval_cash_list


import re

def process_cfflows(cfflows):
    cf_flow_list = [cfflow.text for cfflow in cfflows]
    eval_cf_flow_list = []
    for flow in cf_flow_list:
        # 兆しかない時
        if '兆' in flow and '億' not in flow and '万' not in flow:
            hennkan = flow.replace('兆', '*10**12')
            eval_cf_flow_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in flow and '兆' not in flow and '万' not in flow:
            hennkan = flow.replace('億', '*10**8')
            eval_cf_flow_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in flow and '兆' not in flow and '万' not in flow:
            hennkan = flow.replace('万', '*10**4')
            eval_cf_flow_list.append(eval(hennkan))
        # 兆と億の時
        elif '兆' in flow and '億' in flow:
            hennkan = flow.replace('兆', '*10**12+').replace('億','*10**8')
            eval_cf_flow_list.append(eval(hennkan))
        # 億と万の時
        elif '億' in flow and '万' in flow:
            hennkan = flow.replace('億', '*10**8+').replace('万','*10**4')
            eval_cf_flow_list.append(eval(hennkan))
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_cf_flow_list.append(eval(hennkan))
    return eval_cf_flow_list


def convert_japanese_financial_string(string):
    hennkan = string.replace('兆', '*10**12').replace('億', '*10**8').replace('万', '*10**4')
    return eval(hennkan)



def convert_flow_figure(flow_figure: str) -> int:
        #兆しかない時
        if '兆' in flow_figure and '億' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('兆', '*10**12')
            return eval(hennkan)         
        #億しかない時
        elif '億' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('億', '*10**8')
            return eval(hennkan)             
        #万しかない時
        elif '万' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('万', '*10**4')
            return eval(hennkan)
        #兆と億の時
        elif'兆'in flow_figure and '億' in flow_figure:
            hennkan = flow_figure.replace('兆', '*10**12+').replace('億','*10**8')
            return eval(hennkan)
        #億と万の時
        elif'億'in flow_figure and '万' in flow_figure:
            hennkan = flow_figure.replace('億', '*10**8+').replace('万','*10**4')
            return eval(hennkan)
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", flow_figure)
            return eval(hennkan)


def floatparcent_hennkan(s):
    if '.' in s:
        number_str = s.split('%')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result

    else:
        # 整数部と小数部が分離されていない場合
        integer_part = s.split('%')[0]  # '86' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result
def check_uriagedaka(eval_sale_list):
    result = all(int(eval_sale_list[i]) < int(eval_sale_list[i+1]) for i in range(len(eval_sale_list)-1))
    uriagedaka_stock = []
    if result:
        uriagedaka_stock.append('OK')
    else:
        for i in range(len(eval_sale_list)-1):
            if int(eval_sale_list[i]) >= int(eval_sale_list[i+1]):
                uriagedaka_stock.append(f'NOT (at index {i+1})')
    return uriagedaka_stock


def check_eps(zeps_list):
    result = all(int(zeps_list[i]) < int(zeps_list[i+1]) for i in range(len(zeps_list)-1))
    eps_stock = []
    if result:
        eps_stock.append('OK')
    else:
        for i in range(len(zeps_list)-1):
            if int(zeps_list[i]) >= int(zeps_list[i+1]):
                eps_stock.append(f'NOT (at index {i+1})')
    return eps_stock

def check_eiri(zop_list):
    zop_list = [float(x) for x in zop_list]
    resault = all(10 <= float(zop_list[i]) for i in range(len(zop_list)))
    eiri_stock = []
    if resault:
        eiri_stock.append('OK')
    else:
        for i in range(len(zop_list)):
            if 10 >= float(zop_list[i]):
                eiri_stock.append(f'NOT (at index {i})')
    return eiri_stock

def check_zikosihon(zoption_list):
    oolist = [float(x) for x in zoption_list]
    result = all(40 <= float(oolist[i]) for i in range(len(oolist)))
    zikosihon_stock = []
    if result:
        zikosihon_stock.append('OK')
    else:
        for i in range(len(oolist)):
            if 40 >= float(oolist[i]):
                zikosihon_stock.append(f'NOT (at index {i})')
    return zikosihon_stock


def check_genkin(eval_cash_list):
    result = all(int(eval_cash_list[i]) < int(eval_cash_list[i+1]) for i in range(len(eval_cash_list)-1))
    genkin_stock = []
    if result:
        genkin_stock.append('OK')
    else:
        for i in range(len(eval_cash_list)-1):
            if int(eval_cash_list[i]) >= int(eval_cash_list[i+1]):
                genkin_stock.append(f'NOT (at index {i+1})')
    return genkin_stock




def get_closing_url(code):
    url = f'https://irbank.net/{code}/ir'
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.text,'html.parser')
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    return link



In [ ]:
test__lists

In [155]:
import requests
from cachecontrol import CacheControl 
from cachecontrol.caches import FileCache

session = requests.Session()
# sessionをラップしたcached_sessionを作る。
# キャッシュはファイルとして .webcache ディレクトリ内に保存する。
cached_session = CacheControl(session, cache=FileCache('.webcache'))

response = cached_session.get('https://irbank.net/E31167/results') 

# response.from_cache属性でキャッシュから取得されたレスポンスかどうかを取得できる。
print(f'from_cache: {response.from_cache}') 
print(f'status_code: {response.status_code}')

from_cache: False
status_code: 200


NameError: name 'number_str' is not defined

In [180]:
from logging import getLogger,StreamHandler,DEBUG,ERROR

logger=getLogger(__name__)
handler=StreamHandler()
handler.setLevel(ERROR)
logger.setLevel(DEBUG)
logger.addHandler(handler)
logger.debug('これはデバッグログ')

これはデバッグログ
これはデバッグログ
これはデバッグログ
これはデバッグログ
これはデバッグログ


In [195]:
securities_codes = get_securities_codes(27,28)
for x in securities_codes:
    with open('irbank/csv/securities_codes.pickle','wb') as f:
        pickle.dump(x,f)
    with open('irbank/csv/securities_codes.pickle','rb') as f:
        x=pickle.load(f)
print(x)

5019


time: 0.000744820 [sec]
time: 0.000028849 [sec]
time: 0.000043154 [sec]
time: 0.000023127 [sec]


In [477]:
 df=pd.DataFrame(dd_list)
df.head()
df.shape

(40, 4)

Error: iterable expected, not int

Error: iterable expected, not int

In [764]:
import itertools
with open('irbank/csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

729


In [797]:
existing_dat

NameError: name 'existing_dat' is not defined

In [599]:
with open('irbank/csv/code.pickle', 'rb') as f:
            company_code = pickle.load(f)

In [619]:
del existing_data[0]

709


In [10]:
import time

# 関数の実行時間を測るデコレータ
def tictoc(func):
    def _wrapper(*args, **keywargs):
        start_time = time.time()
        result = func(*args, **keywargs)
        print('time: {:.9f} [sec]'.format(time.time() - start_time))
        return result
    return _wrapper

In [11]:
from functools import cache

@tictoc
@cache  # デコレータを付与
def func(num):
    # なんらかの重い処理
    for _ in range(1000000):
        num += 1
    return num

# 1度目の呼び出し
func(1)

# 2度目の呼び出し
func(1)

time: 0.078004122 [sec]
time: 0.000003099 [sec]


1000001

In [207]:
func(1)

time: 0.000001907 [sec]


1000001

In [794]:
#1
import random
@tictoc
@cache
def get_securities_codes(start_page, end_page):
    url='https://minkabu.jp/financial_item_ranking/dividend_yield?page={}'
    dd_list=[]
    for i in range(start_page, end_page+1):
        target_url=url.format(i)
        r=requests.get(target_url)
        sleep(random.uniform(3, 8)) 
        soup=BeautifulSoup(r.text,'html.parser')
        tab=soup.find('table',class_='md_table rnk_li')
        conts=tab.find_all('tr')
        del conts[0]

        for cont in conts:
            rank=cont.find('span',class_='ranking_no fsize_l').text
            code=cont.find('div',class_='md_sub').text
            name=cont.find('div',class_='fwb w90p').text
            y=cont.find('td',class_='tar cur vamd').text
            yeild= y.strip().replace('%', '')
            dic={
                'ランキング':rank,
                '証券コード':code,
                '企業名':name,
                '配当利回り':yeild,
            }
            
            
            if float(yeild) >= 3.75:
                dd_list.append(dic)
    securities_codes = [d['証券コード'] for d in dd_list]
    #csvに保存まだ更新はできず追加しかされない
    new_data=pd.DataFrame(dd_list)
    if os.path.exists('irbank/csv/20230115ranking.csv'):
                df=pd.read_csv('irbank/csv/20230115ranking.csv')
                df=pd.concat([df, new_data], ignore_index=True)
                
    else:
        df=new_data
    df.to_csv('irbank/csv/20230115ranking.csv',index=None,encoding='utf-8-sig')

    return securities_codes


In [798]:
#1

result = {}
#1ページ目から取得なので値に0はいらない
for page in range(30, 37, 1):
    securities_codes = get_securities_codes(page, page)
    result[page] = securities_codes
    if os.path.exists('irbank/csv/securities_codes.pickle'):
     # ファイルから既存のデータを読み込む
        with open('irbank/csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
        # 既存のデータを更新する
        existing_data.update(result)
        # ファイルに追加されたデータを保存する
        with open('irbank/csv/securities_codes.pickle', 'wb') as f:
            pickle.dump(existing_data, f)
    else:
        del result[0]
        with open('irbank/csv/securities_codes.pickle', 'wb') as f:
            pickle.dump(result, f)

time: 5.357740641 [sec]
time: 6.625604153 [sec]
time: 8.049380302 [sec]
time: 7.945807934 [sec]
time: 5.542266130 [sec]
time: 5.170459032 [sec]
time: 6.947769880 [sec]


In [805]:
import itertools
with open('irbank/csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
            del existing_data[0]
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

713


In [806]:
existing_data

{1: ['9104',
  '9101',
  '9110',
  '9308',
  '1518',
  '1662',
  '2914',
  '1820',
  '9107',
  '5208',
  '5009',
  '5192',
  '5334',
  '7239',
  '8595',
  '3284',
  '7744',
  '8886',
  '2986',
  '6035'],
 2: ['4595',
  '6502',
  '1833',
  '5444',
  '3245',
  '7638',
  '1852',
  '8076',
  '3948',
  '7236',
  '6651',
  '2892',
  '7847',
  '5985',
  '9119',
  '8737',
  '8996',
  '7494',
  '3242',
  '7433'],
 3: ['8093',
  '6156',
  '9268',
  '7088',
  '7762',
  '9434',
  '7480',
  '3465',
  '4249',
  '8075',
  '6291',
  '6918',
  '9282',
  '9381',
  '6461',
  '8150',
  '8999',
  '8893',
  '1822',
  '2975'],
 4: ['4031',
  '1720',
  '5901',
  '5988',
  '4705',
  '1847',
  '7278',
  '8304',
  '7414',
  '6393',
  '1808',
  '9147',
  '2999',
  '4246',
  '6210',
  '7284',
  '3447',
  '5703',
  '7198',
  '1451'],
 5: ['5015',
  '5408',
  '5105',
  '3294',
  '9880',
  '7148',
  '3321',
  '8897',
  '2676',
  '2737',
  '6462',
  '1911',
  '3109',
  '6358',
  '5702',
  '8835',
  '7879',
  '9445',
 

In [543]:
#     with open('irbank/csv/ranking.csv', 'w', newline='') as csvfile:
#         fieldnames = ['ランキング','証券コード','企業名','配当利回り']
#         #辞書型を書き込むためのクラス
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()
#         for d in dd_list:
#             writer.writerow({'ランキング':d['ランキング'],'証券コード': d['証券コード'],'企業名':d['企業名'],'配当利回り':d['配当利回り']})
    


NameError: name 'flattened_code' is not defined

TypeError: list indices must be integers or slices, not str

In [629]:
closing_urls= []
# for code in securities_codes[:3]:
for code in flattened_codes[0:1]:
    closing_urls.append(get_closing_url(code))
print (closing_urls)

time: 0.000003099 [sec]
['/E04236/results']


In [8]:
@tictoc
@cache
def get_closing_url(code):
    url = f'https://irbank.net/{code}/ir'
    r = requests.get(url)
    sleep(random.uniform(3, 8)) 
    soup = BeautifulSoup(r.text,'html.parser')
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    return link


NameError: name 'tictoc' is not defined

In [12]:
#2
#ここからclosing_dict=
import pickle
closing_dict = {}
# for page in range(0, 100, 10):
#     # your code here
#     time.sleep(60) 
for code in flattened_codes[1:300]:
    new_url = get_closing_url(code)
    closing_dict[code] = new_url

# 既存のpickleファイルがある場合
    if os.path.exists('irbank/csv/closing_dict.pickle'):
        # ファイルから既存のデータを読み込む
        with open('irbank/csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)
        # 既存のデータに新しいデータを追加
        ex_data.update(closing_dict)
        # ファイルに追加されたデータを保存する
        with open('irbank/csv/closing_dict.pickle', 'wb') as f:
            pickle.dump(ex_data, f)
    # 初めての保存の場合
    else:
        with open('irbank/csv/closing_dict.pickle', 'wb') as f:
            pickle.dump(closing_dict, f)
            
            


NameError: name 'os' is not defined

In [778]:
#2
with open('irbank/csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)

In [779]:
len(ex_data)

180

In [667]:
from bs4 import BeautifulSoup
import requests 
import re
from time import sleep
import pandas as pd
import time
from tqdm import tqdm
import pickle
import itertools

with open('irbank/csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
#リストを一緒くたに一つに
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

https://irbank.net/E35470/dividend


#flattened_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
with open('irbank/csv/closing_dict.pickle', 'wb') as f:
            existing_data = pickle.load(f)
flatclosing_urls = list(itertools.chain(ex_data.values()))



focus_url=[]
for flatclosing_url in flatclosing_urls[:2]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(2)
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
text=[]
for element in tagget:
    for a in element.find_all('a'):
        a.decompose()
    text.append(element)

729


In [719]:
#3

#ここからfinal_dict
import pickle
final_dict = {}
focus_url=[]
for flatclosing_url in flatclosing_urls[8:10]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(2)
    focus_url.append(url)
    final_dict[flatclosing_url] = focus_url[-1]
    save_pickle(final_dict, 'irbank/csv/final_dict.pickle')


In [720]:
with open('irbank/csv/final_dict.pickle', 'rb') as f:
    ex_data = pickle.load(f)
final_code = list(itertools.chain(ex_data.values()))


In [5]:
flattened_codes

['9104',
 '9101',
 '9110',
 '9308',
 '1518',
 '1662',
 '2914',
 '1820',
 '9107',
 '5208',
 '5009',
 '5192',
 '5334',
 '7239',
 '8595',
 '3284',
 '7744',
 '8886',
 '2986',
 '6035',
 '9104',
 '9101',
 '9110',
 '9308',
 '1518',
 '1662',
 '2914',
 '1820',
 '9107',
 '5208',
 '5009',
 '5192',
 '5334',
 '7239',
 '8595',
 '3284',
 '7744',
 '8886',
 '2986',
 '6035',
 '4595',
 '6502',
 '1833',
 '5444',
 '3245',
 '7638',
 '1852',
 '8076',
 '3948',
 '7236',
 '6651',
 '2892',
 '7847',
 '5985',
 '9119',
 '8737',
 '8996',
 '7494',
 '3242',
 '7433',
 '8093',
 '6156',
 '9268',
 '7088',
 '7762',
 '9434',
 '7480',
 '3465',
 '4249',
 '8075',
 '6291',
 '6918',
 '9282',
 '9381',
 '6461',
 '8150',
 '8999',
 '8893',
 '1822',
 '2975',
 '4031',
 '1720',
 '5901',
 '5988',
 '4705',
 '1847',
 '7278',
 '8304',
 '7414',
 '6393',
 '1808',
 '9147',
 '2999',
 '4246',
 '6210',
 '7284',
 '3447',
 '5703',
 '7198',
 '1451',
 '5015',
 '5408',
 '5105',
 '3294',
 '9880',
 '7148',
 '3321',
 '8897',
 '2676',
 '2737',
 '6462',
 

In [692]:
#3
with open('irbank/csv/final_dict.pickle', 'rb') as f:
    ex_data = pickle.load(f)
final_code = list(itertools.chain(ex_data.values()))

for i in final_code:
    r=requests.get(i)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
    
text=[]
for element in tagget:
    for a in element.find_all('a'):
        a.decompose()
    text.append(element)
target_h2 = []#正規は８項目
for t in text:
    if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t:
        target_h2.append(t)
test_lists=[]
if len(target_h2)==8:
    for i in target_h2:
        bro=i.find_next_sibling()
        ss = bro.find_all('span', class_='text')
        test_lists.append(ss)



In [696]:
tagge

NameError: name 'tagge' is not defined

In [680]:
nw_url

['https://irbank.net/E04236/results', 'https://irbank.net/E04235/results']

In [671]:
import os
import pickle

def save_pickle(data, filepath):
    # 既存のpickleファイルがある場合
    if os.path.exists(filepath):
        # ファイルから既存のデータを読み込む
        with open(filepath, 'rb') as f:
            ex_data = pickle.load(f)
        # 既存のデータに新しいデータを追加
        ex_data.update(data)
        # ファイルに追加されたデータを保存する
        with open(filepath, 'wb') as f:
            pickle.dump(ex_data, f)
    # 初めての保存の場合
    else:
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)


In [695]:
flatclosing_urls

['https://irbank.net/E04236/results', 'https://irbank.net/E04235/results']